# Retention API for TEOTIL3

Outline code for API providing retention/transmission estimates from any location in Norway to the coast. See e-mail from Torstein Finnesand received 21.11.2023 at 07.34 for details.

Needs refining and deploying. Also consider making it standards-compliant using [pygeoapi](https://pygeoapi.io/), as suggested by Kim.

In [1]:
import geopandas as gpd
from fastapi import FastAPI
from fastapi.testclient import TestClient
from pydantic import BaseModel
from shapely.geometry import Point

GPKG_PATH = r"/home/jovyan/shared/common/teotil3/core_data/tidied/teotil3_data.gpkg"

app = FastAPI(docs_url="/", title="TEOTIL3 API")


class Coordinate(BaseModel):
    easting: float
    northing: float


class OutputData(BaseModel):
    regine: str
    totp: float
    tdp: float
    tpp: float
    totn: float
    din: float
    ton: float
    toc: float
    ss: float


@app.post("/transmission_to_coast")
async def transmission_to_coast(coordinate: Coordinate):
    """Estimate transmission of nutrients from any point in Norway to the coast.

    Args
        coordinate: Dict. Co-ordinate pair to query in ETRS89-based UTM Zone 33
                    coordinates (EPSG 25833).

    Returns
        Dict. Transmission factors for the regine containing the specificed
        point.
    """
    # Create point
    point = Point(coordinate.easting, coordinate.northing)

    # Get polygons
    gdf = gpd.read_file(
        GPKG_PATH,
        layer="transmission_to_coast",
        driver="GPKG",
        bbox=[point.x, point.y, point.x, point.y],
    )

    # Find polygon containing point
    for index, row in gdf.iterrows():
        if row["geometry"].contains(point):
            data = row.to_dict()
            del data["geometry"]
            return OutputData(**data)

    return {"error": "No polygon contains the provided point"}


# Testing
client = TestClient(app)

# Langtjern co-ords
east, north = 209438, 6704338
coords = {"easting": east, "northing": north}

# Query API
response = client.post("/transmission_to_coast", json=coords)
print(response.json())

{'regine': '012.CB5D', 'totp': 38.7, 'tdp': 58.8, 'tpp': 20.4, 'totn': 68.3, 'din': 61.4, 'ton': 87.7, 'toc': 50.8, 'ss': 0.1}


In [2]:
response = client.get("/openapi.json")
print(response.json())

{'openapi': '3.1.0', 'info': {'title': 'TEOTIL3 API', 'version': '0.1.0'}, 'paths': {'/transmission_to_coast': {'post': {'summary': 'Transmission To Coast', 'description': 'Estimate transmission of nutrients from any point in Norway to the coast.\n\nArgs\n    coordinate: Dict. Co-ordinate pair to query in ETRS89-based UTM Zone 33\n                coordinates (EPSG 25833).\n\nReturns\n    Dict. Transmission factors for the regine containing the specificed\n    point.', 'operationId': 'transmission_to_coast_transmission_to_coast_post', 'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/Coordinate'}}}, 'required': True}, 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}, '422': {'description': 'Validation Error', 'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}}}}}}, 'components': {'schemas': {'Coordinate': {'properties': {'easting': {'type': '